FE_6 
interaction term
concat 변수 + freq encoding
binary sum 변수 + freq encoding

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')
import random
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import log_loss
import lightgbm as lgb
from datetime import datetime, timedelta

In [ ]:
def tr_vl_split(train_df, num, seed):    
    '''
    train / validation split 함수
    train 에 모든 label이 최소 한번은 등장 & train과 validation의 id는 겹치지 않도록 split함.
    
    train_df : train 데이터
    num : label 당 몇개의 id를 뽑을 것이냐.
    seed = random seed
    
    '''
    
    np.random.seed(seed)
    
    valid_id = []
    vc = train[['id','label']].drop_duplicates()['label'].value_counts()
    temp = list(vc[vc > num].index)
    for a in temp:
        id_list = list(train_df[train_df['label'] == a]['id'])
        valid_id += random.sample(id_list,num)
    
    train_id = list(set(train_df['id']) - set(valid_id))
    
    x_tr_ = train[train['id'].isin(train_id)]
    y_tr_ = train_label[train['id'].isin(train_id)]

    x_vl_ = train[~train['id'].isin(train_id)]
    y_vl_ = train_label[~train['id'].isin(train_id)]
    
    return x_tr_, y_tr_, x_vl_, y_vl_ 

In [ ]:
def target_mode_encoding(col,tr,vl,tst):

    temp = tr.groupby([col])['label'].agg(lambda x:x.value_counts().index[0]).to_dict()
    
    tr[col+'_mode_encoding'] = tr[col].map(temp)
    vl[col+'_mode_encoding'] = vl[col].map(temp)
    tst[col+'_mode_encoding'] = tst[col].map(temp)
    
    return tr, vl, tst

In [ ]:
train = pd.read_pickle('/kaggle/input/15th-data60sec/train_data.pickle/train_data.pickle')
test = pd.read_pickle('/kaggle/input/15th-data60sec/test_data.pickle/test_data.pickle')
train_label = train['label']

var_list = pd.read_csv('/kaggle/input/15th-data60sec/var_type_list.csv')

In [ ]:
etc_var = list(var_list[var_list['type'] == 'etc']['var'])
num_var = list(var_list[var_list['type'] == 'num']['var'])
bin_var = list(var_list[var_list['type'] == 'bin']['var'])
cat_var = list(var_list[var_list['type'] == 'cat']['var'])

var_list = etc_var + num_var + bin_var + cat_var

train_label = train_label[train['time'] >= 10].reset_index(drop=True)
train = train.loc[(train['time'] >= 10),var_list].reset_index(drop=True)
test = test.loc[(test['time'] >= 10),list(set(var_list) - set(['label']))].reset_index(drop=True)


all = pd.concat([train,test],axis=0)

In [ ]:
import_var = ['V3616','V0081','V3324','V3615','V2855','V2859','V1821','V1818','V3098',
 'V3461','V4505','V1820','V2861','V2860','V1819','V3432','V2586','V2854',
 'V4743','V4824','V3103','V2853','V4495','V2076','V4525']

In [ ]:
eng_var = []

In [ ]:
# categorical 변수 중 가까이 위치한 변수들 concat

concat_var = [['V0050','V0051'],['V0469','V0471','V0472'],['V0529','V0530','V0532'],['V0731','V0733','V0735'],['V4502','V4504'],['V4526','V4527']]

for a in tqdm(concat_var):
    
    colnm = '_'.join(a + ['concat'])
    temp = pd.Series(np.repeat('',len(all)))
    
    for col in a:
        temp2 = all[col].astype(str)
        temp = temp.str.cat(temp2)
        
        all[colnm] = pd.factorize(temp)[0]
        
eng_var += [a for a in all.columns if 'concat' in a]

In [ ]:
bin_of_bin = pd.Series(all[bin_var].columns[all[bin_var].max() == 1].to_list())

bin_of_bin_series = pd.Series([int(a.replace('V','')[:2]) for a in list(bin_of_bin)])

# V0000~V10000 / V1001 ~ V2000 / V2001 ~ V3000 / V3001 ~ V4000 / V4001 ~ V5000 으로 만든거
for i in tqdm(range(5)):
    cols = list(bin_of_bin[(bin_of_bin_series >= (i*10 + 1)) & (bin_of_bin_series < (i+1)*10 + 1)])
    all['V_index_'+str(i*10)] = all[cols].sum(axis=1)        

all['V_index_all'] = all[bin_of_bin].sum(axis=1)

In [ ]:
# categorical 변수와 binary concat 변수의 frequency encoding
for col in tqdm(cat_var + [a for a in all.columns if 'concat' in a] + [a for a in all.columns if 'V_index' in a]):
    temp = all.groupby(col)['id'].count().to_dict()
    all[col+'_freq'] = all[col].map(temp)
    del temp
    
eng_var += [a for a in all.columns if 'freq' in a]

In [ ]:
# 변수 중요도에서 상위 5개 변수의 interaction 변수
for i,col1 in tqdm(enumerate(import_var[:5])):
    for i2 in range(i+1,5):
        all[col1+'*'+import_var[:5][i2]] = all[col1] * all[import_var[:5][i2]]
        all[col1+'/'+import_var[:5][i2]] = all[col1] / all[import_var[:5][i2]]
        all[col1+'+'+import_var[:5][i2]] = all[col1] + all[import_var[:5][i2]]
        all[col1+'-'+import_var[:5][i2]] = all[col1] - all[import_var[:5][i2]]
        
eng_var += [a for a in all.columns if '+' in a]
eng_var += [a for a in all.columns if '/' in a]
eng_var += [a for a in all.columns if '*' in a]
eng_var += [a for a in all.columns if '-' in a]

In [ ]:
train = all[~all['label'].isnull()].reset_index(drop=True).drop(columns = ['time'])
test = all[all['label'].isnull()].reset_index(drop=True).drop(columns = ['time','label'])

del all

In [ ]:
x_tr, y_tr, x_vl, y_vl = tr_vl_split(train, 2, seed = 1995)

print('train shape :',x_tr.shape)
print('validation shape :',x_vl.shape)
print('test shape :', test.shape)

In [ ]:
# for col in tqdm(bin_var + cat_var):
#     x_tr, x_vl, test = target_mode_encoding(col,x_tr,x_vl,test)
    
# eng_var += [a for a in list(x_tr) if 'mode' in a]

In [ ]:
%%time

x_tr = x_tr.groupby('id').rolling(window = 5).mean().drop(columns = ['id']).reset_index().drop(columns = ['level_1']).dropna().reset_index(drop=True)
x_vl = x_vl.groupby('id').rolling(window = 5).mean().drop(columns = ['id']).reset_index().drop(columns = ['level_1']).dropna().reset_index(drop=True)

y_tr = x_tr['label']
y_vl = x_vl['label']

test = test.groupby('id').rolling(window = 5).mean().drop(columns = ['id']).reset_index().drop(columns = ['level_1']).dropna().reset_index(drop=True)
test_id = test['id']

var_model = list(set(x_tr.columns) & set(test.columns) - set(['id']))

In [ ]:
%%time

lgb_tr = lgb.Dataset(x_tr[var_model], label=y_tr)
lgb_vl = lgb.Dataset(x_vl[var_model], label=y_vl)

watchlist_1 = [lgb_tr, lgb_vl]
watchlist_2 = [lgb_vl, lgb_tr]

params = {
    "objective": "multiclass",
    "boosting": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    "metric": "multiclass",
    "num_class" : 198
}

lgb_model = lgb.train(params, train_set=lgb_tr, num_boost_round=1000, valid_sets=watchlist_1, verbose_eval=100, early_stopping_rounds=100)

prediction = pd.DataFrame(lgb_model.predict(test[var_model]))

In [ ]:
lgb.plot_importance(lgb_model,max_num_features = 25)

In [ ]:
time_now = datetime.now() + timedelta(hours = 9)
submission_name = str(time_now)[:16] + '_submission.csv'
sub = pd.concat([pd.DataFrame(test_id),prediction],axis=1).groupby('id').mean().reset_index()
sub.to_csv(submission_name,index=False)